In [1]:
suppressPackageStartupMessages({
  if (!require("VennDiagram")) stop("[ERROR] Package 'VennDiagram' is not installed.")
  if (!require("grid")) stop("[ERROR] Package 'grid' is not installed.")
})

In [2]:
# --- 1. CONFIGURATION ---

cell_line   <- "CH12F3"
threshold   <- 3
min_count   <- 5
enlargement <- 0

results_dir <- file.path("/sharedFolder/resultsFolder",
                    paste0("Threshold_", threshold),
                    paste0("Enlargement_", enlargement))

input_file  <- file.path(results_dir, 
                    paste0(enlargement, "_", cell_line, "_SharedHotspots_", threshold, "_rider", min_count, ".bed"))
  if (!file.exists(input_file)) stop(paste0("[ERROR] Input file not found: ", input_file))

output_dir  <- "/sharedFolder/Plots_Euler_Venn_Upset"
    # Check if output directory exists, if not, create it
    if (!dir.exists(output_dir)) {
      cat(paste0("[INFO] Creating output directory: ", output_dir, "\n"))
      dir.create(output_dir, recursive = TRUE)
    } else {
      cat(paste0("[INFO] Output directory already exists.\n"))
    }


[INFO] Output directory already exists.


In [3]:
# --- 2. DATA PROCESSING ---
if (!file.exists(input_file)) stop("[ERROR] Input file not found.")
data <- read.delim(input_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

data[] <- lapply(data, function(x) if(is.character(x)) trimws(x) else x)

metadata_cols <- c("region_id", "chrom", "start", "end")
sample_cols <- setdiff(colnames(data), metadata_cols)
n_samples <- length(sample_cols)

if (n_samples < 1) stop("[ERROR] No sample columns.")

# CHECK LIMITS
if (n_samples > 5) {
  cat(paste0("[WARNING] Detected ", n_samples, " sets. VennDiagram supports a maximum of 5 sets.\n"))
  cat("[WARNING] Skipping Venn plot generation for this dataset.\n")
  quit(save = "no", status = 0) # Exit cleanly without error
}

# Prepare List
venn_list <- list()
for (col in sample_cols) {
  # Robust check: value is "+" AND not NA
  venn_list[[col]] <- which(!is.na(data[[col]]) & data[[col]] == "+")
}

# Colors
if (n_samples <= 3) {
  my_fills <- c("#FF8B94", "#BEDDF1", "#fff2ae")[1:n_samples]
  cat_cols <- c("#FF8B94", "#BEDDF1", "#F6CF71")[1:n_samples]
} else {
  my_fills <- rainbow(n_samples, alpha = 0.7)
  cat_cols <- rainbow(n_samples)
}

[WARNING] Detected 7 sets. VennDiagram supports a maximum of 5 sets.
[WARNING] Skipping Venn plot generation for this dataset.


In [4]:
# --- 3. PLOTTING VENN ---

# Define output path
output_filename <- paste0(enlargement, "_VennPlot_", threshold, "_", min_count, ".png")
output_path <- file.path(output_dir, output_filename)

cat("[INFO] Generating Venn diagram...\n")

tryCatch({
  # Generate the Venn object
  # Note: filename must be NULL to avoid the package writing its own file automatically
  venn_obj <- venn.diagram(
    x = venn_list,
    filename = NULL, 
    fill = my_fills,
    alpha = 0.7,
    lwd = 2,
    lty = "blank",
    cex = 1.0,
    fontfamily = "sans",
    fontface = "bold",
    cat.cex = 1.2,
    cat.col = cat_cols,
    cat.fontfamily = "sans",
    cat.fontface = "bold",
    margin = 0.1
  )
  
  # 1. SAVE TO FILE
  png(output_path, width = 8, height = 8, units = "in", res = 300, bg = "white")
  grid.draw(venn_obj)
  invisible(dev.off())
  
  if (file.exists(output_path)) {
    cat(paste0("[INFO] Saved PNG to: '", output_path, "'\n"))
  }

  # 2. DISPLAY IN JUPYTER
  # Set the plot size for the notebook
  options(repr.plot.width = 8, repr.plot.height = 8)
  
  # Clear the current grid page and draw the object
  grid.newpage()
  grid.draw(venn_obj)
  
}, error = function(e) {
  cat(paste0("[ERROR] Venn generation failed: ", e$message, "\n"))
})

[INFO] Generating Venn diagram...


Warning message in (function (fmt, ...) :
“one argument not used by format 'Incorrect number of elements.'”


[ERROR] Venn generation failed: Incorrect number of elements.
